In [1]:
import twint
import nest_asyncio
import pandas as pd
import json
import networkx as nx

nest_asyncio.apply()

In [6]:
c = twint.Config()
c.Username = 'infoJakarta'
c.Since = "2021-02-01" #batas awal tanggal pengambilan tweet
c.Until = "2021-03-25" #batas akhir tanggal pengambilan tweet
c.Count = True         
c.Retweets = True
c.Stats = True 
c.Show_hashtags = True
c.Hide_output = True
c.Profile_full = True
c.Store_json = True
c.Search = 'vaksin'
c.Output = './data/output_twint_2.json' # save to json
twint.run.Search(c)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 5 Tweets from @infoJakarta.


In [7]:
df = pd.read_json('./data/output_twint_2.json' , lines = True)
# df
# df = df[['conversation_id', 'id','date','time','tweet','mentions','replies_count','retweets_count','likes_count','hashtags','reply_to','link']]
df

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1373529155304706054,1373529155304706048,2021-03-21 13:57:24+07:00,2021-03-21,13:57:24,700,106038219,infojakarta,@InfoJakarta,,...,,,,,,[],,,,
1,1372218371442319360,1372218371442319360,2021-03-17 23:08:48+07:00,2021-03-17,23:08:48,700,106038219,infojakarta,@InfoJakarta,,...,,,,,,[],,,,
2,1372150088248664068,1372150088248664064,2021-03-17 18:37:29+07:00,2021-03-17,18:37:29,700,106038219,infojakarta,@InfoJakarta,,...,,,,,,[],,,,
3,1371706352549040133,1371706352549040128,2021-03-16 13:14:14+07:00,2021-03-16,13:14:14,700,106038219,infojakarta,@InfoJakarta,,...,,,,,,[],,,,
4,1366579485793939456,1366579485793939456,2021-03-02 09:41:53+07:00,2021-03-02,09:41:53,700,106038219,infojakarta,@InfoJakarta,,...,,,,,,[],,,,


In [8]:
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
# from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
import numpy as np
from datetime import datetime, timedelta

In [9]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
# chrome_options.add_argument('headless') # untuk menutup browser

#specify the path to chromedriver.exe
driver = webdriver.Chrome('./chromedriver',chrome_options=chrome_options)

<ipython-input-9-4a1c4cd5d846>:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver',chrome_options=chrome_options)


In [10]:
driver.get('https://www.twitter.com/login')
sleep(2)

username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('dyingrabbit_') # Ubah username sesuai akun pribadi

my_password = getpass()

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

········


In [11]:
data_tweets = df[['conversation_id', 'id','date','time','username','tweet','mentions','replies_count','retweets_count','likes_count','hashtags','reply_to','link']
data_tweets

,conversation_id,id,date,time,username,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,reply_to,link
0,1373529155304706048,1373529155304706054,2021-03-21,13:57:24,infojakarta,Kalau tidak salah baca datanya. Kemarin 20 ma...,"[{'screen_name': 'dkijakarta', 'name': 'pempro...",2,4,5,[],[],https://twitter.com/infojakarta/status/1373529...
1,1372218371442319360,1372218371442319360,2021-03-17,23:08:48,infojakarta,Vaksin lansia di @UnikaAtmaJaya Terbuka untuk...,"[{'screen_name': 'unikaatmajaya', 'name': 'uni...",0,12,13,[],[],https://twitter.com/infojakarta/status/1372218...
2,1372150088248664064,1372150088248664068,2021-03-17,18:37:29,infojakarta,Lansia dengan ktp daerah manapun bisa mendapat...,[],0,5,8,[],[],https://twitter.com/infojakarta/status/1372150...
3,1371706352549040128,1371706352549040133,2021-03-16,13:14:14,infojakarta,"Pelaku industri pariwisata di Jakarta, apakah ...",[],0,0,3,[],[],https://twitter.com/infojakarta/status/1371706...
4,1366579485793939456,1366579485793939456,2021-03-02,09:41:53,infojakarta,"Bergerak pararel. Vaksin jalan, pariwisata be...","[{'screen_name': 'sandiuno', 'name': 'sandiaga...",0,1,2,[],[],https://twitter.com/infojakarta/status/1366579...


In [12]:
def get_list_likes(link):
    list_likes = []
    driver.get(link + '/likes')#open link
    sleep(2) #jeda waktu
    
    #scroll to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(2)
    
    try:
        likes_container = driver.find_element_by_xpath('//div[@aria-label="Timeline: Liked by"]') #element dasar likes
        if likes_container: #jika ada maka
            likes = likes_container.find_elements_by_xpath('//div[@dir="ltr"]/span')
            for like in likes:
                list_likes.append(like.text)
    except:
        return list_likes
    return list_likes



def get_list_retweets(link):
    list_retweets = []
    driver.get(link + '/retweets') #open link
    sleep(2)#jeda waktu
    
    #scroll to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(2)
    
    try:
        retweets_container = driver.find_element_by_xpath('//div[@aria-label="Timeline: Retweeted by"]')
        if retweets_container:
            retweets = retweets_container.find_elements_by_xpath('//div[@dir="ltr"]/span')
            for retweet in retweets:
                list_retweets.append(retweet.text)
    except:
        pass
    return list_retweets

In [16]:
arr_temp = []
for i, row in data_tweets.iterrows():
     arr_temp.append(get_list_likes(row['link']))   

data_tweets['list_likes'] = arr_temp
data_tweets['list_likes']

0    [@FitriRamadhian2, @chiko_jubilee, @FebriYa846...
1    [@Nursanti87, @sermawati, @lalilulouie, @chiko...
2    [@MarwanAdam12, @dhatamapertiwi, @RiaWasta, @N...
3                                                   []
4    [@JupriSetiawan91, @astrraeus, @HaiMagazine, 8...
Name: list_likes, dtype: object

In [14]:
arr_temp = []
for i, row in data_tweets.iterrows():
     arr_temp.append(get_list_retweets(row['link']))

data_tweets['list_retweets'] = arr_temp
data_tweets['list_retweets']

0    [@akoemala, @nutanews184, @liranti, , , , , , ...
1    [@Desta412, @Hamdiees, @wenntyputri, @chiko_ju...
2    [@Hamdiees, @gilangandriani, @chiko_jubilee, ,...
3                                                   []
4    [@faqihiqaf, , , , , , , , , , @kompascom, @sb...
Name: list_retweets, dtype: object

In [32]:
G = nx.Graph()
for edges in data_tweets['list_retweets']:
    for edge in edges:
        G.add_edge('infoJkarta', edge)
nx.draw(G)

conversation_id                                  1373529155304706048
id                                               1373529155304706054
date                                             2021-03-21 00:00:00
time                                                        13:57:24
username                                                 infojakarta
tweet              Kalau tidak salah baca datanya.  Kemarin 20 ma...
mentions           [{'screen_name': 'dkijakarta', 'name': 'pempro...
replies_count                                                      2
retweets_count                                                     4
likes_count                                                        5
hashtags                                                          []
reply_to                                                          []
link               https://twitter.com/infojakarta/status/1373529...
list_likes         [@FitriRamadhian2, @chiko_jubilee, @FebriYa846...
list_retweets      [@akoemala, @nu